# Rocq-ml-toolbox

The goal of this notebook is to present the very basic steps to create and interact with a custom build docker image.

## Installation

First, let's start by installing all the requirements.
In this notebook, we will **USE** docker.

## Docker image

Let's load an Opam config.

In [3]:
from rocq_ml_toolbox.docker.opam_docker import OpamConfig

config = OpamConfig.from_yaml('coq-corn.yaml')

Now, we can construct the associated docker image.

In [4]:

import docker
from rocq_ml_toolbox.docker.opam_docker import OpamDocker

client = docker.from_env()
opam_docker = OpamDocker(config)

opam_docker.install_project(" ".join(config.packages))

Build Image
The following actions will be performed:
=== install 16 packages
  - install atd              3.0.1         [required by atdgen, atdts]
  - install atdgen           3.0.1         [required by elpi]
  - install atdgen-runtime   3.0.1         [required by atdgen]
  - install atdts            3.0.1         [required by elpi]
  - install biniou           1.2.2         [required by atdgen]
  - install camlp-streams    5.0.1         [required by biniou]
  - install coq              9.0.1         [required by coq-corn]
  - install coq-bignums      9.0.0+rocq9.0 [required by coq-corn]
  - install coq-core         9.0.1         [required by coq, coq-elpi]
  - install coq-corn         9.0.0
  - install coq-elpi         3.2.0         [required by coq-corn]
  - install coq-math-classes 9.0.0         [required by coq-corn]
  - install coq-stdlib       9.0.0         [required by coq, coq-bignums]
  - install easy-format      1.3.4         [required by atd]
  - install elpi             3.

[NOTE] Package coq-corn is already installed (current version is 9.0.0).


Now, if you want to save it:

In [ ]:
opam_docker.container.commit(config.name, config.tag)

Let's try to close it, and reload it again:

In [ ]:
opam_docker.close()
opam_docker = OpamDocker(config)

# Do we need to install packages a second time?
opam_docker.install_project(" ".join(config.packages))

Now let's do our usual tasks:

In [5]:
from typing import Dict, List
from collections import defaultdict

from tqdm import tqdm
from rocq_ml_toolbox.inference import PetClient
from rocq_ml_toolbox.parser import RocqParser, Source

opam_docker.start_inference_server()
client = PetClient('127.0.0.1', 5000)
client.connect()
parser = RocqParser(client)

output_sources: Dict[str, List[Source]] = defaultdict(list)
for lib_name in tqdm(opam_docker.list_opam_folder(), desc='Extract sources'):
    output_sources[lib_name] = opam_docker.extract_source_files_from_folder(lib_name)

for lib_name, sources in tqdm(output_sources.items(), desc='Extract TOC'):
    for source in sources:
        toc = parser.extract_toc(source)
        print(toc)
        input('continue?')

ImportError: cannot import name 'PetClient' from 'rocq_ml_toolbox.inference' (unknown location)